# Exploring Queries Against The Sakila Database

The `sakila` database consists of tables, view and triggers, simulating operations for a DVD rental service.
For more details: https://dev.mysql.com/doc/sakila/en/.

**Note**: We use `SQLAlchemy` with `mysqlclient` in order to connect to `MySQL`, since this is more robust
and well-maintained than `mysql-connector-python`. `SQLAlchemy` is a powerful framework, where tables, rows, etc.
can be modelled as Python classes, and queries can be constructed using such objects. We do not use these
properties here.

Extra dependencies required: `SQLAlchemy`, `mysqlclient`, `python-dotenv`, `pandas`.

In [70]:
import pandas as pd

from script_utils.sql import get_database_engine, run_sql_queries, run_sql_query, DatabaseMetaData

Create a `.env` file in this directory, containing the following information about the database and the user:

```text
MYSQL_DATABASE=<name-of-sakila-database>  [default: sakila]
MYSQL_USER=<user-name>
MYSQL_PASSWD=<password>
MYSQL_PORT=<port>                         [default: 3306]
```

For this notebook to work, the user must have `SELECT` privilege on all tables of the database.

In [101]:
# Configuration parameters read from .env file

db_engine = get_database_engine()
db_metadata = DatabaseMetaData(db_engine)

/Users/seeger/git/scripts_and_notebooks/script_utils/sql/metadata.py:11: SAWarning: Did not recognize type 'geometry' of column 'location'
  self._metadata.reflect(engine)


In [72]:
query = """
SELECT customer_id, first_name, last_name
  FROM customer
"""

df = run_sql_query(query, engine=db_engine)
print(f"Number of rows: {df.shape[0]}")
df.head(20)

Number of rows: 599


,customer_id,first_name,last_name
0,1,MARY,SMITH
1,2,PATRICIA,JOHNSON
2,3,LINDA,WILLIAMS
3,4,BARBARA,JONES
4,5,ELIZABETH,BROWN
5,6,JENNIFER,DAVIS
6,7,MARIA,MILLER
7,8,SUSAN,WILSON
8,9,MARGARET,MOORE
9,10,DOROTHY,TAYLOR


In [ ]:
# Determine the number of rows of all tables in the database

query_mask = "SELECT COUNT(*) AS size FROM {}"

table_names = db_metadata.table_names()
maxlen = max(len(table) for table in table_names)
queries = [query_mask.format(table) for table in table_names]
dfs = run_sql_queries(queries, db_engine)

for table, df in zip(table_names, dfs):
    print(f"{table:{maxlen}s}: {df.loc[0, 'size']}")

In [73]:
# Which films appear most often in the inventory of each store?

query = """
SELECT f.title as title, a.address as store, COUNT(*) as number
  FROM inventory as i
  JOIN film as f
    ON f.film_id = i.film_id
  JOIN store as s
    ON s.store_id = i.store_id
  JOIN address as a
    ON a.address_id = s.address_id
 GROUP BY f.film_id, s.store_id
 ORDER BY number DESC
"""

df = run_sql_query(query, db_engine)
df

,title,store,number
0,ACADEMY DINOSAUR,28 MySQL Boulevard,4
1,ACADEMY DINOSAUR,47 MySakila Drive,4
2,ADAPTATION HOLES,28 MySQL Boulevard,4
3,AFFAIR PREJUDICE,47 MySakila Drive,4
4,AIRPORT POLLOCK,28 MySQL Boulevard,4
...,...,...,...
1516,YENTL IDAHO,28 MySQL Boulevard,2
1517,YOUNG LANGUAGE,47 MySakila Drive,2
1518,YOUTH KICK,47 MySakila Drive,2
1519,ZHIVAGO CORE,28 MySQL Boulevard,2


In [ ]:
# What is the range of rental dates?

query1 = """
SELECT MIN(rental_date), MAX(rental_date)
  FROM rental
"""

query2 = """
SELECT MIN(return_date), MAX(return_date)
  FROM rental
 WHERE return_date IS NOT NULL
"""

dfs = run_sql_queries([query1, query2], db_engine)
print(dfs[0])
print(dfs[1])

In [ ]:
query1 = """
SELECT rental_date, return_date
  FROM rental
 WHERE return_date IS NOT NULL
 ORDER BY return_date
"""

query2 = """
SELECT rental_date
  FROM rental
 WHERE return_date IS NULL
 ORDER BY rental_date
"""

dfs = run_sql_queries([query1, query2], db_engine)
dfs[0]

In [ ]:
dfs[1]

In [ ]:
query = """
SELECT rental_date, inventory_id, customer_id
  FROM rental
 WHERE return_date IS NULL
 ORDER BY rental_date
"""
run_sql_query(query, db_engine)

The dates in `rental` are odd. We have `rental_date` and `return_date`, the latter can be missing, which probably means the item has not been returned yet.

* For records with `return_date` present, the `'rental_date` to `return_date` spans ranges between 2005-05-25 and 2005-09-02
* For records with `return_date` missing, one `rental_date` is 2005-08-21, followed by 181 rows with `rental_date` equal to "2006-02-14 15:16:03". These records are for different items and customers.

It looks like `rental_date` has been corrupted for these 181 rows. They should be filtered out.

Once this is done, `rental` contains 15860 rows with `return_date` present in the date range 2005-05-25 to 2005-09-02 (4.5 months), and one row with `return_date` missing and `rental_date` 2005-08-21.

As the first more interesting query, for a given timestamp, we'd like a table of films which are in stock at that time in one of the stores, and how many items of each film are currently in stock.

In [74]:
# Timestamp for which we like to check in-stock status
query_datetime = "2005-06-01 12:10:05"

# First, list all inventory items which are currently rented out
query_mask = """
SELECT inventory_id, rental_id, rental_date, return_date
  FROM rental
 WHERE rental_date <= '{0}' AND (return_date IS NULL OR return_date > '{0}')
"""

df = run_sql_query(query_mask.format(query_datetime), db_engine)
df

,inventory_id,rental_id,rental_date,return_date
0,1711,3,2005-05-24 23:03:39,2005-06-01 22:12:39
1,2452,4,2005-05-24 23:04:41,2005-06-03 01:43:41
2,2079,5,2005-05-24 23:05:21,2005-06-02 04:33:21
3,4443,11,2005-05-25 00:09:02,2005-06-02 20:56:02
4,3049,15,2005-05-25 00:39:22,2005-06-03 03:30:22
...,...,...,...,...
682,2725,1153,2005-05-31 21:36:44,2005-06-10 01:26:44
683,2732,1154,2005-05-31 21:42:09,2005-06-08 16:40:09
684,2048,1155,2005-05-31 22:17:11,2005-06-04 20:27:11
685,460,1156,2005-05-31 22:37:34,2005-06-01 23:02:34


The previous query lists inventory items which are rented out, we need to take the difference of all inventory
with this result.

ANSI SQL provides set operations `UNION`, `UNION ALL`, `INTERSECT`, `EXCEPT`, the last one being the difference.
However, MySQL does not support `INTERSECT`, `EXCEPT`.

But we can do this differently. As a subgoal, we'd like to select all inventory IDs in the desired difference,
a table with a single column. This can be joined with `inventory` to obtain the remaining fields. One idea is to
use something like `WHERE inventory_id NOT IN (...)`, where the subquery is the one above. However, the subquery
result can be large, and this can be inefficient.

We can use a `LEFT JOIN` of `inventory` with the query above (projected to `inventory_id`), selecting those rows
where the right `inventory_id` is `NULL`. Since ID values are unique, these rows in a left join correspond
exactly to those where the left `inventory_id` is not contain on the right side. This is the set difference.

In [102]:
# How does this work?
# - The query labeled as ir selects the inventory which is rented out at query_datetime
# - instock_inventory (CTE) is the set difference between inventory and ir, so contains
#   inventory not rented out at query_datetime.
#   This is done via a LEFT JOIN, which contains rows with ir.inventory_id IS NULL for
#   every inventory not contained in ir.
#   MySQL does not support set difference explicitly.
# - Finally, we join with film to obtain titles and count in-stock inventory per film
#   and store. We also join with inventory, since the instock_inventory CTE only
#   contains inventory IDs

query_mask = """
WITH
instock_inventory AS (
SELECT DISTINCT ia.inventory_id
  FROM inventory AS ia
  LEFT JOIN (SELECT inventory_id
               FROM rental
              WHERE rental_date <= '{0}' AND (return_date IS NULL OR return_date > '{0}')) AS ir
    ON ia.inventory_id = ir.inventory_id
 WHERE ir.inventory_id IS NULL
)

SELECT f.film_id, i.store_id, f.title, COUNT(*) AS num_in_stock
  FROM inventory AS i
  JOIN instock_inventory AS ii
    ON i.inventory_id = ii.inventory_id
  JOIN film AS f
    ON i.film_id = f.film_id
 GROUP BY f.film_id, i.store_id
 ORDER BY f.film_id, i.store_id
"""

df_instock = run_sql_query(query_mask.format(query_datetime), db_engine)
df_instock

,film_id,store_id,title,num_in_stock
0,1,1,ACADEMY DINOSAUR,3
1,1,2,ACADEMY DINOSAUR,4
2,2,2,ACE GOLDFINGER,3
3,3,2,ADAPTATION HOLES,3
4,4,1,AFFAIR PREJUDICE,3
...,...,...,...,...
1501,998,2,ZHIVAGO CORE,1
1502,999,1,ZOOLANDER FICTION,2
1503,999,2,ZOOLANDER FICTION,2
1504,1000,1,ZORRO ARK,4


In [78]:
# Sanity test

test_mask = """
SELECT i.store_id, f.film_id, f.title, COUNT(*) AS num
  FROM inventory AS i
  JOIN (SELECT inventory_id
          FROM rental
         WHERE rental_date <= '{0}' AND (return_date IS NULL OR return_date > '{0}')) as ir
    ON i.inventory_id = ir.inventory_id
  JOIN film AS f
    ON i.film_id = f.film_id
 GROUP BY f.film_id, i.store_id
 ORDER BY f.film_id, i.store_id
"""

df_rented = run_sql_query(test_mask.format(query_datetime), db_engine)

test_query = """
SELECT i.store_id, f.film_id, f.title, COUNT(*) AS num
  FROM inventory as i
  JOIN film AS f
    ON i.film_id = f.film_id
 GROUP BY i.store_id, f.film_id
 ORDER BY f.film_id, i.store_id
"""

df_total = run_sql_query(test_query, db_engine)

names = ["store_id", "film_id", "num"]
df1_instock = df_instock.rename(columns={"num_in_stock": "num"})[names]
df1_rented = df_rented[names]
df1_mtotal = df_total[names]
df1_mtotal.loc[:, "num"] *= (-1)

test_df = pd.concat([df1_instock, df1_rented, df1_mtotal]).groupby(["store_id", "film_id"]).sum()
test_df[test_df["num"] != 0]

,,num
store_id,film_id,


In [79]:
run_sql_query("SELECT * FROM staff", db_engine)

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-02-15 03:57:16
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,None,2006-02-15 03:57:16


In [80]:
run_sql_query("SELECT * FROM store", db_engine)

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [ ]:
run_sql_query("SELECT film_id, title, rental_duration, rental_rate FROM film", db_engine)

In [81]:
query = """
SELECT r.rental_id, r.rental_date, f.rental_duration, DATE_ADD(r.rental_date, INTERVAL f.rental_duration DAY) as due_date, r.return_date
  FROM rental AS r
  JOIN inventory AS i
    ON r.inventory_id = i.inventory_id
  JOIN film as f
    ON i.film_id = f.film_id
"""

run_sql_query(query, db_engine)

,rental_id,rental_date,rental_duration,due_date,return_date
0,4863,2005-07-08 19:03:15,6,2005-07-14 19:03:15,2005-07-11 21:29:15
1,11433,2005-08-02 20:13:10,6,2005-08-08 20:13:10,2005-08-11 21:35:10
2,14714,2005-08-21 21:27:43,6,2005-08-27 21:27:43,2005-08-30 22:26:43
3,972,2005-05-30 20:21:07,6,2005-06-05 20:21:07,2005-06-06 00:36:07
4,2117,2005-06-17 20:24:00,6,2005-06-23 20:24:00,2005-06-23 17:45:00
...,...,...,...,...,...
16039,711,2005-05-29 03:49:03,3,2005-06-01 03:49:03,2005-05-31 08:29:03
16040,1493,2005-06-15 21:50:32,3,2005-06-18 21:50:32,2005-06-17 01:02:32
16041,6712,2005-07-12 13:24:47,3,2005-07-15 13:24:47,2005-07-20 09:35:47
16042,9701,2005-07-31 07:32:21,3,2005-08-03 07:32:21,2005-08-01 05:07:21


Next, we would like to understand how amounts in `payment` are determined.

In [82]:
query = """
SELECT r.rental_date,
       r.return_date,
       DATEDIFF(DATE(r.return_date), DATE(r.rental_date)) AS days_rented,
       f.rental_duration,
       f.rental_rate,
       p.amount,
       f.rental_rate + GREATEST(DATEDIFF(DATE(r.return_date), DATE(r.rental_date)), f.rental_duration) - f.rental_duration AS exp_amount
  FROM rental AS r
  JOIN inventory AS i
    ON r.inventory_id = i.inventory_id
  JOIN film as f
    ON i.film_id = f.film_id
  JOIN payment AS p
    ON r.rental_id = p.rental_id
 WHERE r.return_date IS NOT NULL
"""
df = run_sql_query(query, db_engine)
df

,rental_date,return_date,days_rented,rental_duration,rental_rate,amount,exp_amount
0,2005-07-08 19:03:15,2005-07-11 21:29:15,3,6,0.99,0.99,0.99
1,2005-08-02 20:13:10,2005-08-11 21:35:10,9,6,0.99,3.99,3.99
2,2005-08-21 21:27:43,2005-08-30 22:26:43,9,6,0.99,3.99,3.99
3,2005-05-30 20:21:07,2005-06-06 00:36:07,7,6,0.99,1.99,1.99
4,2005-06-17 20:24:00,2005-06-23 17:45:00,6,6,0.99,0.99,0.99
...,...,...,...,...,...,...,...
15856,2005-05-29 03:49:03,2005-05-31 08:29:03,2,3,4.99,4.99,4.99
15857,2005-06-15 21:50:32,2005-06-17 01:02:32,2,3,4.99,4.99,4.99
15858,2005-07-12 13:24:47,2005-07-20 09:35:47,8,3,4.99,9.99,9.99
15859,2005-07-31 07:32:21,2005-08-01 05:07:21,1,3,4.99,4.99,4.99


In [ ]:
df[df["amount"] != df["exp_amount"]]

Here is what is going on:
* `payment.amount` is `film.rental_rate` plus 1 dollar per extra day beyond `film.rental_duration` (if any)
* Number of extra days are determined from the dates only, the times do not play a role. This is why we use
  `DATEDIFF(DATE(r.return_date), DATE(r.rental_date))`
* It is weird that `payment.payment_date` is the same as `rental.rental_date`. We cannot know the number of
  overdue days up front, unless the customer explicitly rents the movie for longer up front.
* It would make more sense to charge `film.rental_rate` at `rental.rental_date`, and then the extra overdue fee
  at `rental.return_date` when the item is returned.

Next, for each customer, we would like to list all films rented out, their due dates, and overdue fees (if any). This is again relative to a specific timestamp.

In [98]:
# Timestamp for which we like to list films rented, due dates, and overdue fees
query_datetime = "2005-06-01 12:10:05"

query_mask = """
SELECT CONCAT(c.last_name, ", ", c.first_name) as customer_name,
       f.title, 
       r.rental_date,
       DATE_ADD(r.rental_date, INTERVAL f.rental_duration DAY) as due_date,
       GREATEST(DATEDIFF(DATE('{0}'), DATE(r.rental_date)), f.rental_duration) - f.rental_duration as overdue_fee
  FROM rental as r
  JOIN inventory as i
    ON r.inventory_id = i.inventory_id
  JOIN film as f
    ON i.film_id = f.film_id
  JOIN customer as c
    ON r.customer_id = c.customer_id
 WHERE r.rental_date <= '{0}' AND (r.return_date IS NULL OR r.return_date > '{0}')
 ORDER BY c.last_name, c.first_name, due_date
"""

df = run_sql_query(query_mask.format(query_datetime), db_engine)
print(f"Today: {query_datetime}")
df

Today: 2005-06-01 12:10:05


,customer_name,title,rental_date,due_date,overdue_fee
0,"ABNEY, RAFAEL",POCUS PULP,2005-05-28 19:14:09,2005-06-03 19:14:09,0
1,"ADAM, NATHANIEL",GATHERING CALENDAR,2005-05-27 21:17:08,2005-05-31 21:17:08,1
2,"ADAM, NATHANIEL",NOON PAPI,2005-05-30 00:27:57,2005-06-04 00:27:57,0
3,"ALLEN, SHIRLEY",STREAK RIDGEMONT,2005-05-29 16:03:03,2005-06-05 16:03:03,0
4,"ALVAREZ, CHARLENE",INDEPENDENCE HOTEL,2005-05-27 22:47:39,2005-06-01 22:47:39,0
...,...,...,...,...,...
682,"WOODS, FLORENCE",CHARADE DUFFEL,2005-05-31 03:45:26,2005-06-03 03:45:26,0
683,"WREN, TYLER",WYOMING STORM,2005-05-27 00:47:35,2005-06-02 00:47:35,0
684,"WYMAN, BRIAN",VAMPIRE WHALE,2005-05-26 10:18:27,2005-05-30 10:18:27,2
685,"YANEZ, LUIS",GILMORE BOILED,2005-05-29 17:35:50,2005-06-03 17:35:50,0


What is the sum of overdue fees per customer?

In [99]:
query_mask = """
WITH
overdue_fees_customer_film AS (
SELECT c.customer_id,
       CONCAT(c.last_name, ", ", c.first_name) as customer_name,
       GREATEST(DATEDIFF(DATE('{0}'), DATE(r.rental_date)), f.rental_duration) - f.rental_duration as overdue_fee
  FROM rental as r
  JOIN inventory as i
    ON r.inventory_id = i.inventory_id
  JOIN film as f
    ON i.film_id = f.film_id
  JOIN customer as c
    ON r.customer_id = c.customer_id
 WHERE r.rental_date <= '{0}' AND (r.return_date IS NULL OR r.return_date > '{0}')
)

SELECT customer_name,
       SUM(overdue_fee) AS total_fees
  FROM overdue_fees_customer_film
 GROUP BY customer_id
 ORDER BY total_fees DESC, customer_name
"""

df = run_sql_query(query_mask.format(query_datetime), db_engine)
print(f"Today: {query_datetime}")
df

Today: 2005-06-01 12:10:05


,customer_name,total_fees
0,"SEWARD, LESLIE",8.0
1,"GAMEZ, CLARENCE",7.0
2,"HANSEN, DELORES",6.0
3,"WEINER, RONALD",5.0
4,"AUSTIN, ALMA",4.0
...,...,...
406,"WINDHAM, DARREN",0.0
407,"WOFFORD, VIRGIL",0.0
408,"WREN, TYLER",0.0
409,"YANEZ, LUIS",0.0
